In [28]:
import cv2
import sys
import numpy as np 

# how to measure distances 
the distances are distorted due to curvature of the earth 

## types of energy we can sense 
- magnetic filed
- electric field
## 2 models 
- waves 
- particles (photons)
E = planck length * speed of light / wave length 

long wave -> lower energy, harder to measuer, sensor has to be more sensitive and also will get more noise 

small waves -> higher energy, easier to detect but easier to bump on smt 

## bodies EM energy 

- spectral energy 
- W/m^2*wavelength - a lot of energy as it gets hotter 
- higher energt more into blue 
- emitents [0,1] 
- some of the peaks of energy will be distorted due to some interference 
- black box model 

## visability spectrum 

- UV light - visable by birds and insects 
- eyes reflect infrared - used for eye tracker 
- if we want to skip some material we can use 
    - bandpass filter 
    - coat the sensor with the material
     
## why we dont wanna perfect step signal of the color 
- the color regions overlap 
- we do not percive 'perfect blue' or 'perfect green'

- for light: Yellow (R+G), Cyan = (B+G) ,Magenda = (R+B)
- the more 'blue' we add to we see more red/violet 

## particles 

- when we multuply recieved signal by the function of 'particles' we will recieve the ammount of what particles are there e.g. on the other planet 

## shadow zone and recursion zone
- important to understnd while looking at the atmosphere 

## libraries 
- measuring vegetation - e.g. how many % of healthy plants we have 
- a lot of chlorophila - red/green
- reflect a lot of infrared 

## NDVI
- normalized difference vegetation index 
- diff between mirrored infrader and red signal - big diff for healthy vegetatiion
- divide by the sum of inf red and red 

In [4]:
# PCA - feeding multilayer gausian distribution !!! check it

# 1. measure vegetation
#### 1. Obtain data 
#### 2. Visualize the data
#### 3. Segment the data 
#### 4. Find object 

Lidar - energy in the phase of laser beam is heating the the object 
and then reflecting the light
- usually data in the form of peaks 
- intensity of the laser 
- National agriculture data, they have infrared data 

# 1. Get data 
- elevation data (close to 1m as we can get, more accurate the better)
- RGB NIR data (napi 1m)
- shape that on the map (vector data)

### Websites

- open topography 
- usgs earth exploer 
- https://viewer.nationalmap.gov/basic/

Lidar data is big but really high resolution (even in the resolution of inches)

In [1]:
import rasterio as rio
import matplotlib.pylab as plt

In [2]:
# for jpeg2000 we sometimes have to change it for tip
np_ds = rio.open('oxford.tif')

In [9]:
np_ds.crs

CRS.from_epsg(26916)

In [ ]:
np_ds.af

In [3]:
%matplotlib widget

In [10]:
im = np_ds.read(out_shape = (4,512,512))

In [11]:
from rasterio.plot import show as ds_show

In [17]:
# ds_show(np_ds)

In [19]:
# faster, not rendering 
plt.figure('napi-prev')
# we have to show just 3 bands 
plt.imshow(im.transpose(1,2,0)[:,:,(0,1,2)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# transform.affine
scale_x = np_ds.affine.a
scale_y = np_ds.affine.e
map_x = np_ds.affine.c
map_y = np_ds.affine.f
# print scale 
# print position - map


AttributeError: 'DatasetReader' object has no attribute 'affine'

!['end'](img/endlesson1.png)
!['end2'](img/endlesson2.png)

In [ ]:
np_ds.bounds

In [ ]:
figure('map-pre')

dynamic range of and eye
!['week4'](img/week4.png)
!['week](img/week44.png)

- we are more sensitive for the values in particular range 
- it is depending on the e.g. brightness/darkness of the room/place where we are
- we adjust the range which we can see by pupile dilegts ( rozszerzanie sie soczewki)
- in AI we also do not store all memory, we are more tend to guess - we approximate y' to y gamma,
- y - energy values 
!['week](img/week41.png)
- 255 e.g. is gonna be the values of gamma which we sense and picked up 
- sometimes we have to go to the metadata file to actually know what is the data - on the image the y^gamma approximation is being used
!['week](img/week42.png)




Contrast and brightness

gain and bias

- we have a space where we can see the colors 
- this is the are where we see but it is not being particularly correct, it is distorted by gammma transformation 
- due to the not evenly space , we cannot see evenly well in different colors/spaces
- we have some areas 

HDR 
!['week](img/week45.png)
- take many images and then combine them together 
- as human beings, we can see all details looking at the sun,
- but when taking phot with camra we cannnot do this so welll
- we analyze it in our eyes and also our brain 
- high special resolution 
- 16 bits -> 0 - 65535 -> better precision 
- but when representinf floating point numbers, they are still not nonlinear,
- the numbers in 16 bits are scaled int - values are from 0 - 1 and then they are devided by 65535

In [24]:
np_ds.colorinterp

(<ColorInterp.red: 3>,
 <ColorInterp.green: 4>,
 <ColorInterp.blue: 5>,
 <ColorInterp.alpha: 6>)

In [25]:
# dtype unisgnt int 8bits 
np_ds.meta 

{'driver': 'GTiff',
 'dtype': 'uint8',
 'nodata': None,
 'width': 9910,
 'height': 12450,
 'count': 4,
 'crs': CRS.from_epsg(26916),
 'transform': Affine(0.6, 0.0, 687720.0,
        0.0, -0.6, 4381818.0)}

In [26]:
8//10 # int division 

0

In [29]:
inf = im.astype(np.int)
ndvi = (inf[3]-inf[2])/(inf[3]+inf[2])

In [32]:
plt.figure('ndvi')
plt.imshow(ndvi,vmin=0)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
# we can also look at outside the image 
im2 = np_ds.read(out_shape = (4,512,512), window = ((-100,100),(5000,6000)),boundless = True )

In [35]:
plt.figure('window')
plt.imshow(im2,vmin=0)
plt.colorbar()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: Invalid shape (4, 512, 512) for image data

In [38]:
bounds = np_ds.window_bounds([(-100,900),(5000,6000)])
# range of lattitude and longitude

In [39]:
im = np_ds.read()

# recognition on the image 

!['week](img/week47.png)
- image has bands / channels (e.g. RGB, infrared),
- 3D arrays - tensors 
- columns - spacial locations 
- channels - attributes of the lacation ('features') 
- but how to recognize whethere or no a group of attributes is a car/building ect.
- we can see if its building but also e.g. what is its hight,
- classification problem - recognition, regression problem - measurements
- we have a big image, we slide through the image 

We can try to do it in many ways in all the channels at the same time:
- window operation (neighbors)
- point operation 
- we can treat it as a R^C dimensional space but kinda consider it as one point,

Preprocessing

- e.g. presegmentation, separating on the bigger pieces, bigger than a pixel but smaller than the object we are looking for 
- clustering based on the channels 
- e.g. watershed, superpixels, 

- pixel4D 